# 🏭 CORTEX PRODUCTION — Mode Batch

**Traitement automatique de toutes les vidéos**

---

### ⚡ Quick Start
1. Exécuter **CELLULE 2** (Setup)
2. Exécuter **CELLULE 3** (Tir Massif)
3. *(Optionnel)* Exécuter **CELLULE 4** (Rapport)

### ⚠️ Prérequis
- Vidéos dans `DRIVE_EXODUS_V2/00_CORTEX_HQ/IN_VIDEO_SOURCE/`
- Clé API Gemini dans Colab Secrets (`GEMINI_API_KEY`)

In [ ]:
#@title CELLULE 2 — Setup Complet (One-Click)
#@markdown Installation + Montage + Configuration en une seule étape

# ═══════════════════════════════════════════════════════════════
# INSTALLATION
# ═══════════════════════════════════════════════════════════════
!pip install google-generativeai opencv-python-headless -q

import os
import sys
import json
from datetime import datetime

# ═══════════════════════════════════════════════════════════════
# MONTAGE GOOGLE DRIVE
# ═══════════════════════════════════════════════════════════════
from google.colab import drive, userdata
drive.mount('/content/drive')

# ═══════════════════════════════════════════════════════════════
# CONFIGURATION CHEMINS
# ═══════════════════════════════════════════════════════════════
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"  #@param {type:"string"}
INPUT_DIR = f"{DRIVE_ROOT}/00_CORTEX_HQ/IN_VIDEO_SOURCE"
OUTPUT_DIR = f"{DRIVE_ROOT}/00_CORTEX_HQ/OUT_MANIFEST"
CODEBASE_DIR = f"{DRIVE_ROOT}/00_CORTEX_HQ/CODEBASE"

# Créer dossiers si nécessaires
for d in [INPUT_DIR, OUTPUT_DIR]:
    os.makedirs(d, exist_ok=True)

# ═══════════════════════════════════════════════════════════════
# CLÉ API GEMINI
# ═══════════════════════════════════════════════════════════════
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
except:
    GEMINI_API_KEY = input("🔑 Clé API Gemini: ")

os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

# ═══════════════════════════════════════════════════════════════
# IMPORT MODULE CORTEX
# ═══════════════════════════════════════════════════════════════
sys.path.insert(0, CODEBASE_DIR)

from EXO_00_CORTEX import (
    call_gemini,
    get_video_metadata,
    finalize_output,
    CortexLogger
)

print("")
print("═" * 60)
print("✅ SETUP COMPLET")
print("═" * 60)
print(f"📂 Input:  {INPUT_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")
print(f"🔑 API:    Configurée")

In [ ]:
#@title CELLULE 3 — Tir Massif (Batch Processing)
#@markdown Traitement automatique de TOUTES les vidéos dans IN_VIDEO_SOURCE

# ═══════════════════════════════════════════════════════════════
# CONFIGURATION BATCH
# ═══════════════════════════════════════════════════════════════
MODEL_NAME = "gemini-2.5-flash-preview-05-20"  #@param ["gemini-2.5-flash-preview-05-20", "gemini-2.0-flash", "gemini-1.5-pro", "gemini-1.5-flash"]
SKIP_EXISTING = True  #@param {type:"boolean"}

# Extensions vidéo supportées
VIDEO_EXTENSIONS = ('.mp4', '.mov', '.avi', '.mkv', '.webm')

# ═══════════════════════════════════════════════════════════════
# SCAN DES VIDÉOS
# ═══════════════════════════════════════════════════════════════
videos = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(VIDEO_EXTENSIONS)]

# Filtrer les vidéos déjà traitées si demandé
if SKIP_EXISTING:
    existing_outputs = os.listdir(OUTPUT_DIR) if os.path.exists(OUTPUT_DIR) else []
    videos_to_process = []
    for v in videos:
        video_base = v.rsplit('.', 1)[0]
        if not any(video_base in out for out in existing_outputs):
            videos_to_process.append(v)
    skipped = len(videos) - len(videos_to_process)
    if skipped > 0:
        print(f"⏭️ {skipped} vidéo(s) déjà traitée(s) (skip)")
    videos = videos_to_process

print("")
print("═" * 60)
print("🏭 CORTEX PRODUCTION — TIR MASSIF")
print("═" * 60)
print(f"🎯 {len(videos)} vidéo(s) à traiter")
print(f"🤖 Modèle: {MODEL_NAME}")
print("═" * 60)

if len(videos) == 0:
    print("\n⚠️ Aucune vidéo à traiter!")
    print(f"   Déposez vos vidéos dans: {INPUT_DIR}")
else:
    # ═══════════════════════════════════════════════════════════════
    # TRAITEMENT EN SÉRIE
    # ═══════════════════════════════════════════════════════════════
    logger = CortexLogger(verbose=False)
    results = []
    start_batch = datetime.now()

    for i, video in enumerate(videos, 1):
        print(f"\n[{i}/{len(videos)}] 🎬 {video}")
        start_video = datetime.now()

        try:
            video_path = f"{INPUT_DIR}/{video}"

            # Métadonnées
            metadata = get_video_metadata(video_path, logger)

            # Analyse Gemini
            raw_result = call_gemini(
                video_path=video_path,
                metadata=metadata,
                logger=logger,
                model_name=MODEL_NAME
            )

            if raw_result:
                # Sauvegarde
                video_base = video.rsplit('.', 1)[0]
                output_name = f"PRODUCTION_PLAN_{video_base}.json"
                output_path = f"{OUTPUT_DIR}/{output_name}"

                success = finalize_output(raw_result, output_path, logger)

                if success:
                    elapsed = (datetime.now() - start_video).total_seconds()
                    results.append({
                        "video": video,
                        "status": "✅",
                        "output": output_name,
                        "time_seconds": elapsed
                    })
                    print(f"   ✅ Succès → {output_name} ({elapsed:.1f}s)")
                else:
                    results.append({"video": video, "status": "❌", "error": "Échec sauvegarde"})
                    print(f"   ❌ Échec sauvegarde")
            else:
                results.append({"video": video, "status": "❌", "error": "Échec analyse Gemini"})
                print(f"   ❌ Échec analyse Gemini")

        except Exception as e:
            results.append({"video": video, "status": "❌", "error": str(e)})
            print(f"   ❌ Erreur: {e}")

    # ═══════════════════════════════════════════════════════════════
    # RÉSUMÉ FINAL
    # ═══════════════════════════════════════════════════════════════
    total_time = (datetime.now() - start_batch).total_seconds()
    success_count = sum(1 for r in results if r["status"] == "✅")
    fail_count = len(results) - success_count

    print("\n")
    print("═" * 60)
    print("📊 RÉSUMÉ DE PRODUCTION")
    print("═" * 60)
    print(f"Total:   {len(results)} vidéo(s)")
    print(f"Succès:  {success_count} ✅")
    print(f"Échecs:  {fail_count} ❌")
    print(f"Temps:   {total_time:.1f}s ({total_time/60:.1f} min)")
    print("─" * 60)

    for r in results:
        if r["status"] == "✅":
            print(f"  ✅ {r['video']} → {r['output']}")
        else:
            print(f"  ❌ {r['video']} — {r.get('error', 'Erreur inconnue')}")

    print("═" * 60)

    if success_count == len(results):
        print("🎉 MISSION ACCOMPLIE — PRODUCTION TERMINÉE")
    elif success_count > 0:
        print(f"⚠️ PRODUCTION PARTIELLE — {fail_count} échec(s)")
    else:
        print("❌ PRODUCTION ÉCHOUÉE")

In [ ]:
#@title CELLULE 4 — Rapport de Batch (Optionnel)
#@markdown Génère un rapport JSON détaillé du batch

if 'results' in dir() and results:
    # Générer rapport
    report = {
        "batch_info": {
            "timestamp": datetime.now().isoformat(),
            "model": MODEL_NAME,
            "input_dir": INPUT_DIR,
            "output_dir": OUTPUT_DIR
        },
        "statistics": {
            "total_videos": len(results),
            "successful": success_count,
            "failed": fail_count,
            "success_rate": f"{(success_count/len(results)*100):.1f}%" if results else "0%",
            "total_time_seconds": total_time
        },
        "details": results
    }

    # Sauvegarder rapport
    report_name = f"BATCH_REPORT_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    report_path = f"{OUTPUT_DIR}/{report_name}"

    with open(report_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)

    print("📋 RAPPORT DE BATCH")
    print("═" * 60)
    print(f"✅ Rapport généré: {report_name}")
    print(f"   Chemin: {report_path}")
    print("")
    print(json.dumps(report, indent=2, ensure_ascii=False))
else:
    print("⚠️ Aucun résultat de batch disponible")
    print("   Exécutez d'abord la CELLULE 3 (Tir Massif)")